In [1]:
from datasets import load_dataset
ds = load_dataset("/data/hf_datasets/colpali_train_set/", split="train")


/home/jinaai/miniconda3/envs/rerank_vlm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds

Dataset({
    features: ['image', 'image_filename', 'query', 'answer', 'source', 'options', 'page', 'model', 'prompt', 'answer_type'],
    num_rows: 118195
})

In [3]:
ds[:5]

{'image': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1000x1600>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1700x2200>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1475x1850>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=1709x2293>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=1784x2287>],
 'image_filename': ['images/1810.07757_2.jpg',
  'data/scrapped_pdfs_split/pages_extracted/energy_train/1d09a977-063b-463f-a897-2eda99c1a4f6.pdf/page_9.jpg',
  'data/scrapped_pdfs_split/pages_extracted/energy_train/51b52f38-78e6-4b2d-81c6-b029b02a91cf.pdf/page_414.jpg',
  '0fd47b51ae9248ef36669b8619b1223f268edae3e7a44ac1e6cebbbfaaf69f96',
  'b335cfb9d442f8925ea41a064cb445a5395577f2345d52a64f69f4d5e02ce50c'],
 'query': ['Comparing panels a, b, c, and d, which statement best describes the data variance?',
  'What is the duration of the course mentioned in the image?',
  'What is the primary purpose of the PTC in lithium batteries?',
  'What is the

In [4]:
ds[:5]["image"]

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1000x1600>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1700x2200>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1475x1850>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=1709x2293>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=1784x2287>]

In [5]:
type(ds)

datasets.arrow_dataset.Dataset

In [2]:
# show images ds[0]["image"]

import numpy as np
from PIL import Image

img = ds[0]["image"]
img.show()

display-im6.q16: unable to open X server `' @ error/display.c/DisplayImageCommand/412.


### merge lora to colpali

In [3]:
import torch
from peft import PeftModel, PeftConfig
from colpali_engine.models import ColQwen2, ColQwen2Processor

# 加载基础模型
base_model = ColQwen2.from_pretrained(
    "/data/hf_models/colqwen2-base",  
    torch_dtype=torch.bfloat16,
    device_map="cuda:5"
)

# 加载 LoRA 模型
peft_model_id = "/data/hf_models/colqwen2-v0.1/checkpoint-3694"
model = PeftModel.from_pretrained(base_model, peft_model_id)

# 合并权重
merged_model = model.merge_and_unload()

# 保存合并后的模型
merged_model.save_pretrained("/data/hf_models/colqwen2-v0.1-merged")



`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.29s/it]


In [12]:
import torch
# from PIL import Image
from peft import PeftModel, PeftConfig
from colpali_engine.models import ColQwen2, ColQwen2Processor

In [4]:
# import torch
# from PIL import Image
# from colpali_engine.models import ColQwen2, ColQwen2Processor

# 使用合并后的模型和处理器
model = ColQwen2.from_pretrained(
    "/data/hf_models/colqwen2-v0.1-merged",
    torch_dtype=torch.bfloat16,
    device_map="cuda:4"
).eval()

peft_model_id = "/data/hf_models/colqwen2-v0.1"
processor = ColQwen2Processor.from_pretrained(peft_model_id)




### lora load dynamic

In [2]:
import torch
from peft import PeftModel, PeftConfig
from colpali_engine.models import ColQwen2, ColQwen2Processor

# 加载基础模型
base_model = ColQwen2.from_pretrained(
    "/data/hf_models/colqwen2-base",  
    torch_dtype=torch.bfloat16,
    device_map="cuda:7"
)
# 加载 LoRA 配置
peft_model_id = "/data/hf_models/colqwen2-v0.1"
config = PeftConfig.from_pretrained(peft_model_id)

# 创建 PeftModel
model = PeftModel.from_pretrained(base_model, peft_model_id)
model.eval()

# 加载处理器
processor = ColQwen2Processor.from_pretrained(peft_model_id)


`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.26s/it]


In [3]:
images = ds[:5]["image"]
queries = ds[:1]["query"]

# Process the inputs
batch_images = processor.process_images(images).to(model.device)
batch_queries = processor.process_queries(queries).to(model.device)

# Forward pass
with torch.no_grad():
    image_embeddings = model(**batch_images)
    query_embeddings = model(**batch_queries)

scores = processor.score_multi_vector(query_embeddings, image_embeddings)

In [4]:
query_embeddings.shape, image_embeddings.shape, scores.shape

(torch.Size([1, 42, 128]), torch.Size([5, 755, 128]), torch.Size([1, 5]))

In [5]:
scores


tensor([[1.4922, 1.5234, 1.4297, 1.4062, 1.3906]])

#### batch handle

In [5]:

# 遍历前5个样本
for i in range(5):
    sample = ds[i]
    
    # 直接使用数据集中的图像对象
    image = sample['image']
    # query = sample['query']
    query = ds[0]['query']
    
    # 处理输入
    batch_images = processor.process_images([image]).to(model.device)
    batch_queries = processor.process_queries([query]).to(model.device)
    
    # 推理
    with torch.no_grad():
        image_embeddings = model(**batch_images)
        query_embeddings = model(**batch_queries)
    
    scores = processor.score_multi_vector(query_embeddings, image_embeddings)
    
    print(f"Sample {i}:")
    print(f"Query: {query}")
    print(f"Score: {scores.item()}")

    # print(query_embeddings.shape, image_embeddings.shape, scores.shape)
    del image_embeddings, query_embeddings
    # torch.cuda.empty_cache()

Sample 0:
Query: Comparing panels a, b, c, and d, which statement best describes the data variance?
Score: 0.0
Sample 1:
Query: Comparing panels a, b, c, and d, which statement best describes the data variance?
Score: 0.0
Sample 2:
Query: Comparing panels a, b, c, and d, which statement best describes the data variance?
Score: 0.0
Sample 3:
Query: Comparing panels a, b, c, and d, which statement best describes the data variance?
Score: 0.0
Sample 4:
Query: Comparing panels a, b, c, and d, which statement best describes the data variance?
Score: 0.0


In [23]:
# print(query_embeddings,image_embeddings)
processor = ColQwen2Processor.from_pretrained(peft_model_id)
scores = processor.score_multi_vector(query_embeddings, image_embeddings)
scores

tensor([[0.]])

In [3]:
print(torch.cuda.is_available())
!nvcc -V


True
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Feb__7_19:32:13_PST_2023
Cuda compilation tools, release 12.1, V12.1.66
Build cuda_12.1.r12.1/compiler.32415258_0


In [ ]:
subset_ds = ds.select(range(128))
subset_ds

Dataset({
    features: ['image', 'image_filename', 'query', 'answer', 'source', 'options', 'page', 'model', 'prompt', 'answer_type'],
    num_rows: 128
})

In [5]:
from contextlib import contextmanager

@contextmanager
def temporary_tensor():
    try:
        yield
    finally:
        torch.cuda.empty_cache()

socre_list = []
batch_size = 32
for batch in subset_ds.iter(batch_size=batch_size):
    images = batch['image']
    queries = batch['query']
    
    batch_images = processor.process_images(images).to(model.device)
    batch_queries = processor.process_queries(queries).to(model.device)
    
    with torch.no_grad(), temporary_tensor():
        image_embeddings = model(**batch_images)
        query_embeddings = model(**batch_queries)
    
    scores = processor.score_multi_vector(query_embeddings, image_embeddings)
    # 清理掉计算好的显存
    # del image_embeddings, query_embeddings
    # torch.cuda.empty_cache()
    socre_list.append(scores)
    
    # 处理scores...

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.79 GiB. GPU 7 has a total capacity of 23.65 GiB of which 6.14 GiB is free. Including non-PyTorch memory, this process has 17.51 GiB memory in use. Of the allocated memory 15.31 GiB is allocated by PyTorch, and 637.28 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)